# Semantic KernelとOpenBnB MCPサーバーの統合

このノートブックでは、Semantic Kernelを使用して、実際のOpenBnB MCPサーバーを利用し、MCPStdioPluginを使ってAirbnbの宿泊施設を検索する方法を示します。LLMアクセスにはAzure AI Foundryを使用します。環境変数の設定については、[セットアップレッスン](/00-course-setup/README.md)を参照してください。


## 必要なパッケージをインポート


In [ ]:
# Import cell - Updated imports
import json
import os
import asyncio
import subprocess
import sys


from dotenv import load_dotenv
from IPython.display import display, HTML
from typing import Annotated

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStdioPlugin
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent

## MCPプラグイン接続の作成

MCPStdioPluginを使用して[OpenBnB MCPサーバー](https://github.com/openbnb-org/mcp-server-airbnb)に接続します。このサーバーは、@openbnb/mcp-server-airbnbパッケージを通じてAirbnbの検索機能を提供します。


## クライアントの作成

このサンプルでは、Azure AI Foundry を使用して LLM にアクセスします。環境変数が正しく設定されていることを確認してください。


## 環境設定

Azure OpenAI の設定を構成します。以下の環境変数が設定されていることを確認してください：
- `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_API_KEY`


In [ ]:
# Creating the Client cell - Updated for Azure
load_dotenv()

# Azure OpenAI configuration
# Ensure these environment variables are set:
# - AZURE_OPENAI_CHAT_DEPLOYMENT_NAME
# - AZURE_OPENAI_ENDPOINT
# - AZURE_OPENAI_API_KEY (optional if using DefaultAzureCredential)

chat_completion_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    # Optional - will use DefaultAzureCredential if not set
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

## OpenBnB MCP統合の理解

このノートブックは、実際のAirbnb検索機能を提供する**本物のOpenBnB MCPサーバー**に接続します。

### 動作の仕組み:

1. **MCPStdioPlugin**: MCPサーバーとの標準入出力通信を使用
2. **実際のNPMパッケージ**: `@openbnb/mcp-server-airbnb`をnpxでダウンロードして実行
3. **ライブデータ**: AirbnbのAPIから実際の物件データを返却
4. **機能の発見**: エージェントがMCPサーバーから利用可能な機能を自動的に発見

### 利用可能な機能:

OpenBnB MCPサーバーは通常以下の機能を提供します:
- **search_listings** - 場所や条件でAirbnb物件を検索
- **get_listing_details** - 特定の物件の詳細情報を取得
- **check_availability** - 特定の日付の空き状況を確認
- **get_reviews** - 物件のレビューを取得
- **get_host_info** - 物件ホストの情報を取得

### 前提条件:

- **Node.js** がシステムにインストールされていること
- MCPサーバーパッケージをダウンロードするための**インターネット接続**
- **NPX** が利用可能であること (Node.jsに付属)

### 接続のテスト:

MCPサーバーを手動でテストするには以下を実行してください:
```bash
npx -y @openbnb/mcp-server-airbnb
```

これによりOpenBnB MCPサーバーがダウンロードされ起動し、その後Semantic Kernelが接続して実際のAirbnbデータを取得します。


## OpenBnB MCPサーバーでエージェントを実行する

これから、OpenBnB MCPサーバーに接続するAIエージェントを実行し、ストックホルムで2人の大人と1人の子供向けの実際のAirbnb宿泊施設を検索します。検索条件を変更したい場合は、`user_inputs`リストを自由に編集してください。


In [ ]:
user_inputs = [
    "Find Airbnb in Stockholm for 2 adults 1 kid",
]


async def main():
    """Main function to run the MCP-enabled agent with real OpenBnB server using Azure OpenAI"""

    try:
        print("🚀 Starting with Azure OpenAI...")
        
        # Verify environment variables
        print("🔍 Checking Azure environment variables...")
        required_vars = ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME", "AZURE_OPENAI_ENDPOINT", "AZURE_OPENAI_API_KEY"]
        for var in required_vars:
            if os.getenv(var):
                print(f"✅ {var} is set")
            else:
                print(f"❌ {var} is NOT set")
        
        print("\n🔧 Creating MCP Plugin...")
        
        # Create MCP plugin connection to real OpenBnB server
        # Based on the GitHub repo, the server doesn't need special env vars
        async with MCPStdioPlugin(
            name="AirbnbSearch",
            description="Search for Airbnb accommodations using OpenBnB MCP server",
            command="npx",
            args=["-y", "@openbnb/mcp-server-airbnb"],
        ) as airbnb_plugin:

            print("✅ MCP Plugin created and connected")
            
            # Wait a moment for the server to fully initialize
            await asyncio.sleep(2)
            
            # Try to list available tools
            try:
                tools = await airbnb_plugin.get_tools()
                print(f"🔧 Available tools: {[tool.name for tool in tools]}")
            except Exception as e:
                print(f"⚠️ Could not list tools: {str(e)}")

            # Create the Azure OpenAI service with proper configuration
            print("\n🤖 Creating Azure OpenAI service...")
            service = AzureChatCompletion(
                deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
                endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
                api_key=os.getenv("AZURE_OPENAI_API_KEY"),
            )
            
            # Create agent with the service instance
            agent = ChatCompletionAgent(
                service=service,
                name="AirbnbAgent",
                instructions="""You are an Airbnb search assistant. Use the available functions to search for properties. 
                Format results in a clear HTML table with columns for property name, price, rating, and link.""",
                plugins=[airbnb_plugin],
            )

            print("✅ Agent created with Azure OpenAI")

            # Process each user input
            thread: ChatHistoryAgentThread | None = None

            for user_input in user_inputs:
                print(f"\n🔍 User: {user_input}")
                
                try:
                    # Use the simpler get_response method
                    response = await agent.get_response(messages=user_input, thread=thread)
                    thread = response.thread
                    
                    # Process the response text
                    response_text = str(response)
                    
                    # Remove any markdown code blocks around HTML
                    response_text = response_text.replace('```html', '').replace('```', '')
                    
                    # Display the result
                    print(f"🤖 {response.name}: {response_text[:200]}..." if len(response_text) > 200 else response_text)
                    
                    # If response contains HTML table, display it properly
                    if '<table' in response_text.lower():
                        # Add CSS styling for better table rendering
                        table_css = """
                        <style>
                            .airbnb-results table {
                                border-collapse: collapse;
                                width: 100%;
                                margin: 10px 0;
                            }
                            .airbnb-results th, .airbnb-results td {
                                border: 1px solid #ddd;
                                padding: 8px;
                                text-align: left;
                            }
                            .airbnb-results th {
                                background-color: #f2f2f2;
                                font-weight: bold;
                            }
                            .airbnb-results tr:nth-child(even) {
                                background-color: #f9f9f9;
                            }
                            .airbnb-results a {
                                color: #1976d2;
                                text-decoration: none;
                            }
                            .airbnb-results a:hover {
                                text-decoration: underline;
                            }
                        </style>
                        """
                        html_output = f'{table_css}<div class="airbnb-results">{response_text}</div>'
                        display(HTML(html_output))
                    else:
                        # Display as regular text if no table
                        display(HTML(f'<div class="airbnb-results">{response_text}</div>'))
                        
                except Exception as e:
                    print(f"❌ Error processing user input: {str(e)}")
                    import traceback
                    traceback.print_exc()
                
            # Cleanup
            if thread:
                await thread.delete()
                print("🧹 Thread cleaned up")
                
    except Exception as e:
        print(f"❌ Main error: {str(e)}")
        import traceback
        traceback.print_exc()

# Run the main function
print("🚀 Starting MCP Agent...")
await main()
print("✅ Done!")

# 概要
おめでとうございます！あなたは、Model Context Protocol (MCP) を使用して現実世界の宿泊施設検索と統合するAIエージェントを成功裏に構築しました。

## 使用した技術:
- Semantic Kernel - Azure OpenAIを活用したインテリジェントエージェントの構築
- Azure AI Foundry - LLM機能とチャット補完のため
- MCP (Model Context Protocol) - 標準化されたツール統合のため
- OpenBnB MCP Server - 実際のAirbnb検索機能のため
- Node.js/NPX - 外部MCPサーバーの実行のため

## 学んだこと:
- MCP統合: Semantic Kernelエージェントを外部MCPサーバーに接続する方法
- リアルタイムデータアクセス: 実際のAirbnb物件をライブAPIで検索する方法
- プロトコル通信: エージェントとMCPサーバー間でstdio通信を使用する方法
- 機能の発見: MCPサーバーから利用可能な機能を自動的に発見する方法
- ストリーミングレスポンス: 関数呼び出しをリアルタイムでキャプチャしログに記録する方法
- HTMLレンダリング: スタイル付きテーブルやインタラクティブな表示でエージェントの応答をフォーマットする方法

## 次のステップ:
- 追加のMCPサーバーを統合する (天気、フライト、レストランなど)
- MCPとA2Aプロトコルを組み合わせたマルチエージェントシステムを構築する
- 独自のデータソース用にカスタムMCPサーバーを作成する
- セッションをまたいだ永続的な会話メモリを実装する
- MCPサーバーのオーケストレーションとともにエージェントをAzure Functionsにデプロイする
- ユーザー認証と予約機能を追加する



---

**免責事項**:  
この文書は、AI翻訳サービス [Co-op Translator](https://github.com/Azure/co-op-translator) を使用して翻訳されています。正確性を期すよう努めておりますが、自動翻訳には誤りや不正確な表現が含まれる可能性があります。元の言語で記載された原文を公式な情報源としてご参照ください。重要な情報については、専門の人間による翻訳を推奨します。本翻訳の利用に起因する誤解や誤認について、当社は一切の責任を負いません。
